In [1]:
import pandas as pd
import numpy as np

from Models.grid_search_utils import GridSearch as Gs
from pandas import DataFrame

In [2]:
LGBMRanker = Gs.load_model(name="../Models/BlackBox/lmart/saved_models/LGBMRanker")

In [3]:
FIGSReg = Gs.load_model(name="../Models/GlassBox/figs/saved_models/FIGSReg")

In [4]:
EBMReg = Gs.load_model(name="../Models/GlassBox/ebm/saved_models/EBMReg")

In [5]:
def predict_rank(model, X:DataFrame, top_k:int=15):
    y_pred = model.predict(np.asarray(X.iloc[:,5:].values))
    y_pred = pd.DataFrame(y_pred, index=X.index, columns=["lambdas"])
    dt_final = pd.merge(X, y_pred, left_index=True, right_index=True)
    return dt_final.sort_values("lambdas",ascending=False)[["kId","lambdas","w_score","binned_score"]].head(top_k)

In [6]:
test_set = pd.read_csv("../outputs/scores/0_dataset_ts.csv")

In [7]:
test_set[test_set["binned_score"]==5]

,qId,kId,score,w_score,binned_score,fitness_edu_basic,fitness_edu_bonus,fitness_city,fitness_age,fitness_exp_basic,fitness_exp_bonus,fitness_lang_basic,fitness_lang_bonus,fitness_comp_basic,fitness_comp_bonus,fitness_know_basic,fitness_knowl_bonus
57,28,2175,7.212015,0.782022,5,1.0,0.5,0.591516,1.0,1.0,0.0,1.00,0.0,1.000000,0.069972,1.000000,0.050527
68,36,2870,6.575330,0.707767,5,1.0,0.5,0.000000,1.0,1.0,0.0,1.00,0.3,0.540278,0.195031,1.000000,0.040021
117,36,2920,6.898534,0.743216,5,1.0,0.5,0.000000,1.0,1.0,0.0,1.00,0.0,1.000000,0.204969,1.000000,0.193565
340,93,7423,7.513142,0.793657,5,1.0,0.5,0.000000,1.0,1.0,0.5,1.00,0.0,1.000000,0.013142,1.000000,0.500000
388,111,8964,6.200000,0.725730,5,1.0,0.0,0.000000,1.0,1.0,0.0,0.70,0.0,1.000000,0.000000,1.000000,0.500000
393,40,3239,6.385326,0.719469,5,1.0,0.0,0.000000,1.0,1.0,0.0,1.00,0.0,1.000000,0.040472,1.090580,0.254274
447,25,1971,6.702516,0.707707,5,1.0,0.5,0.000000,1.0,1.0,0.0,1.00,0.0,1.000000,0.192896,1.000000,0.009620
523,66,5365,7.064140,0.733164,5,1.0,0.5,0.000000,1.0,1.0,0.5,1.00,0.0,1.000000,0.010494,1.000000,0.053646
625,32,2509,7.632830,0.723769,5,1.0,0.5,0.000000,1.0,1.0,0.5,0.85,0.0,1.150000,0.260330,1.000000,0.372499
780,17,1377,6.272797,0.735387,5,1.0,0.0,0.000000,1.0,1.0,0.0,1.00,0.0,1.000000,0.051531,1.000000,0.221266


In [8]:
job = test_set[test_set["qId"]==88]

# Perfect rank

In [9]:
job[["kId","w_score","binned_score"]].sort_values("w_score",ascending=False).head(15)

,kId,w_score,binned_score
830,7002,0.632660,4
2069,7016,0.603639,4
222,7038,0.603638,4
802,7054,0.593627,4
1069,7012,0.591609,4
374,7023,0.590890,4
1900,7036,0.588896,4
577,6999,0.579023,4
2532,7001,0.565812,4
2756,7043,0.537330,3


# LGBMRanker

In [10]:
predict_rank(LGBMRanker, job)

,kId,lambdas,w_score,binned_score
404,6995,5.113272,0.525126,3
2532,7001,4.397461,0.565812,4
802,7054,4.340311,0.593627,4
577,6999,4.164576,0.579023,4
2756,7043,4.162132,0.537330,3
374,7023,3.955279,0.590890,4
1069,7012,3.863234,0.591609,4
2017,7017,2.682893,0.456558,3
548,7039,2.282603,0.452935,3
830,7002,1.882201,0.632660,4


# EBM Regressor

In [11]:
predict_rank(EBMReg, job)

,kId,lambdas,w_score,binned_score
2756,7043,0.598617,0.537330,3
2532,7001,0.596431,0.565812,4
404,6995,0.591878,0.525126,3
802,7054,0.580422,0.593627,4
1069,7012,0.572542,0.591609,4
374,7023,0.554686,0.590890,4
830,7002,0.548093,0.632660,4
577,6999,0.535854,0.579023,4
1900,7036,0.535381,0.588896,4
1676,7027,0.532336,0.521310,3


# FIGSRegressor

In [12]:
predict_rank(FIGSReg, job)

,kId,lambdas,w_score,binned_score
2532,7001,0.634002,0.565812,4
404,6995,0.585974,0.525126,3
2756,7043,0.578530,0.537330,3
802,7054,0.578530,0.593627,4
577,6999,0.569552,0.579023,4
1900,7036,0.550037,0.588896,4
374,7023,0.549907,0.590890,4
1069,7012,0.549907,0.591609,4
2017,7017,0.540288,0.456558,3
830,7002,0.535456,0.632660,4
